In [38]:
import pandas as pd
import requests
import datetime
import pytz
import random
import numpy as np
TZ = pytz.timezone('America/Chicago')
pd.options.display.max_colwidth = 250

In [39]:
df = pd.read_json(requests.get('http://lin1pr1ap5:5000/getall').text)
df.drop_duplicates(inplace=True)
df['display_time'] = pd.to_datetime(df.Time)
time = list(df.display_time.dt.strftime("%Y-%m-%d %H:%M"))
seconds = list(np.random.choice(range(0,60),df.shape[0]))
df['display_time'] = pd.to_datetime(time, format='%Y-%m-%d %H:%M')
#df['display_time'] = pd.to_datetime(["%s:%02d" % t for t in zip(time, seconds)])
df['TimeDelta'] = pd.to_datetime(datetime.datetime.now(tz=TZ).strftime("%Y-%m-%d %H:%M:%S"))  - df.display_time
df = df[df.TimeDelta <= datetime.timedelta(hours=1)] ###take last hour of stream
df[["display_time","TimeDelta","Notes"]].sort_values("display_time", ascending=False).head(50)


,display_time,TimeDelta,Notes


# Some Stats:
- 24 Million events created and updated by NAST 2017
- Assuming 2.5 people tied to the load, that's 60 Million emails
- 20 seconds per email, that over 25 thousand hours of waste or over $1 Million per year of effort

In [35]:
###last hour of values
df.groupby("PredictedIntent").agg({'Notes':'count'})

,Notes
PredictedIntent,
Accessorials,3
Appt_Request,10
Appt_Set,158
ETA_Request,140
ETA_Set,145
Other,662
Problem,332
Requirements,5


In [36]:
df.loc[df.PredictedIntent == "Problem",["display_time","Notes","PredictedIntent"]].head(20)

,display_time,Notes,PredictedIntent
40535,2018-03-21 14:23:00,Dr must take pictrs inside of trlr to validate chute and cleanliness compliancy .detail is better. Pics must be sent Driscolls@chrobinson.com by1500 CST day of loading or DR may be bnced.** customer reqrmnt email GROEJEF w/ questions -Load# reqd...,Problem
40540,2018-03-21 14:23:00,Driver is getting loaded but he is given JB HUNT BOL Please advise if that is okay,Problem
40548,2018-03-21 14:23:00,Shipment has been Delayed: Weather or Natural Disaster Related - Shipment del,Problem
40549,2018-03-21 14:23:00,please confirm if driver has been loaded and that he is aware we can't change the delivery apt time,Problem
40566,2018-03-21 14:23:00,Shipment has been Delayed: Weather or Natural Disaster Related - Shipment del,Problem
40570,2018-03-21 14:23:00,Shipment has been Delayed: Weather or Natural Disaster Related - Shipment del,Problem
40585,2018-03-21 14:23:00,Shipment has been Delayed: Weather or Natural Disaster Related - Shipment del,Problem
40590,2018-03-21 14:23:00,Shipment has been Delayed: Other carrier related - Delivery delay caused by a,Problem
40591,2018-03-21 14:23:00,Shipment has been Delayed: Weather or Natural Disaster Related - Shipment del,Problem
40593,2018-03-21 14:23:00,Shipment has been Delayed: Weather or Natural Disaster Related - Shipment del,Problem


In [37]:
###update cycle time is 1 minute
import time
import numpy as np
# ------------------- new cell ---------------------#

from bokeh.models.sources import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.plotting import figure, curdoc
from bokeh.layouts import row
from bokeh.io import output_notebook, show, push_notebook
# ------------------- new cell ---------------------#

output_notebook()
# ------------------- new cell ---------------------#

hover = HoverTool(tooltips=[("Time",
                             "@display_time"),
                            ("Predicted Class","@classification"), ("Real Time Count",
                                                "@count")])

price_plot  = figure(
    plot_width=1000,
    plot_height=600,
    x_axis_type='datetime',
    tools=[hover],
    title="Real-Time E-log Operational Issues")


data_a = ColumnDataSource(dict(time=[], display_time=[], count=[],classification=[]))
data_b = ColumnDataSource(dict(time=[], display_time=[], count=[],classification=[]))
data_c = ColumnDataSource(dict(time=[], display_time=[], count=[],classification=[]))
data_d = ColumnDataSource(dict(time=[], display_time=[], count=[],classification=[]))
data_e = ColumnDataSource(dict(time=[], display_time=[], count=[],classification=[]))
data_f = ColumnDataSource(dict(time=[], display_time=[], count=[],classification=[]))
data_g = ColumnDataSource(dict(time=[], display_time=[], count=[],classification=[]))
data_h = ColumnDataSource(dict(time=[], display_time=[], count=[],classification=[]))


price_plot.line(
    source=data_a, x='time', y='count', color='blue', legend='Other')
price_plot.line(
    source=data_b, x='time', y='count', color='red', legend='ETA_Set')
price_plot.line(
    source=data_c, x='time', y='count', color='orange', legend='ETA_Request')
price_plot.line(
    source=data_d, x='time', y='count', color='purple', legend='Requirements')
price_plot.line(
    source=data_e, x='time', y='count', color='black', legend='Problem')
price_plot.line(
    source=data_f, x='time', y='count', color='pink', legend='Accessorials')
price_plot.line(
    source=data_g, x='time', y='count', color='yellow', legend='Appt_Set')
price_plot.line(
    source=data_h, x='time', y='count', color='green', legend='Appt_Request')

price_plot.xaxis.axis_label = "Time"
price_plot.yaxis.axis_label = "Event Count"
price_plot.title.text = "Real Time Event Log Counts"
price_plot.legend.location = "top_left"


handle = show(price_plot, notebook_handle=True)

df_stable = 10 #minutes stable to show
df_varying = 1 #minutes to show real time update effect
steptimelive = 1 #seconds for refresh on live

###grab last stable period and iterate over final minute
df = pd.read_json(requests.get('http://lin1pr1ap5:5000/getall').text)
df.drop_duplicates(inplace=True)
df['display_time'] = pd.to_datetime(df.Time)
timemin = list(df.display_time.dt.strftime("%Y-%m-%d %H:%M"))
df["plot_time"] = pd.to_datetime(timemin,format = "%Y-%m-%d %H:%M")
seconds = list(np.random.choice(range(0,60),df.shape[0]))
df['display_time'] = pd.to_datetime(["%s:%02d" % t for t in zip(timemin, seconds)])
df['TimeDelta'] = pd.to_datetime(datetime.datetime.now(tz=TZ).strftime("%Y-%m-%d %H:%M:%S"))  - df.display_time

###window to show
df = df[(df.TimeDelta <= datetime.timedelta(minutes=df_stable))]
df["TimeDeltaDisplay"] = df.display_time.max() - df.display_time

###build up the static plots from last stable time
plotit = df[(df.TimeDeltaDisplay > datetime.timedelta(minutes=df_varying))]
new_price = plotit.groupby(["plot_time","PredictedIntent"], as_index=False).agg({"Notes":"count"})
timesplot = df.plot_time.unique()

for t in timesplot:
    ts = pd.to_datetime(str(t)) 
    ts = ts.strftime('%Y-%m-%d %H:%M:%S')
    tmp = new_price[new_price.plot_time <= t]
    tmp = tmp.groupby("PredictedIntent",as_index=False).agg({'Notes':'sum'})
    try:
        a = tmp[tmp["PredictedIntent"] == 'Other']["Notes"].values[0]
    except:
        a = 0
    try:
        b = tmp[tmp["PredictedIntent"] == 'ETA_Set']["Notes"].values[0]
    except:
        b = 0

    try:
        c = tmp[tmp["PredictedIntent"] == 'ETA_Request']["Notes"].values[0]
    except:
        c = 0
    try:
        d = tmp[tmp["PredictedIntent"] == 'Requirements']["Notes"].values[0]
    except:
        d = 0
    try:
        e = tmp[tmp["PredictedIntent"] == 'Problem']["Notes"].values[0]
    except:
        e = 0
    try:
        f = tmp[tmp["PredictedIntent"] == 'Accessorials']["Notes"].values[0]
    except:
        f = 0
    try:
        g = tmp[tmp["PredictedIntent"] == 'Appt_Set']["Notes"].values[0]
    except:
        g = 0
    try:
        h = tmp[tmp["PredictedIntent"] == 'Appt_Request']["Notes"].values[0]
    except:
        h = 0


    data_a.stream(
        dict(
        time=[t],
        display_time=[ts],
        classification=['Other'],
        count=[a]))
    
    data_b.stream(
        dict(
        time=[t],
        display_time=[ts],
        classification=['ETA_Set'],
        count=[b]))
    
    data_c.stream(
        dict(
        time=[t],
        display_time=[ts],
        classification=['ETA_Request'],
        count=[c]))
    
    data_d.stream(
        dict(
        time=[t],
        display_time=[ts],
        classification=['Requirements'],
        count=[d]))
    data_e.stream(
        dict(
        time=[t],
        display_time=[ts],
        classification=['Problem'],
        count=[e]))
    data_f.stream(
        dict(
        time=[t],
        display_time=[ts],
        classification=['Accessorials'],
        count=[f]))
    data_g.stream(
        dict(
        time=[t],
        display_time=[ts],
        classification=['Appt_Set'],
        count=[g]))
    data_h.stream(
        dict(
        time=[t],
        display_time=[ts],
        classification=['Appt_Request'],
        count=[h]))

    push_notebook(handle=handle)
    time.sleep(0.25)  


for i in list(range(steptimelive,df_varying*60 + steptimelive,steptimelive)):
    tconv = pd.to_datetime(str(t)) 
    tstep = tconv + datetime.timedelta(seconds=i)
    ts = tstep.strftime('%Y-%m-%d %H:%M:%S')
    plotit = df[(df.TimeDeltaDisplay > datetime.timedelta(seconds=df_varying*60 - i))]
    ####ok now we start real time streaming
    tmp = plotit.groupby(["PredictedIntent"], as_index=False).agg({"Notes":"count"})

    try:
        a = tmp[tmp["PredictedIntent"] == 'Other']["Notes"].values[0]
    except:
        a = 0
    try:
        b = tmp[tmp["PredictedIntent"] == 'ETA_Set']["Notes"].values[0]
    except:
        b = 0

    try:
        c = tmp[tmp["PredictedIntent"] == 'ETA_Request']["Notes"].values[0]
    except:
        c = 0
    try:
        d = tmp[tmp["PredictedIntent"] == 'Requirements']["Notes"].values[0]
    except:
        d = 0
    try:
        e = tmp[tmp["PredictedIntent"] == 'Problem']["Notes"].values[0]
    except:
        e = 0
    try:
        f = tmp[tmp["PredictedIntent"] == 'Accessorials']["Notes"].values[0]
    except:
        f = 0
    try:
        g = tmp[tmp["PredictedIntent"] == 'Appt_Set']["Notes"].values[0]
    except:
        g = 0
    try:
        h = tmp[tmp["PredictedIntent"] == 'Appt_Request']["Notes"].values[0]
    except:
        h = 0


    data_a.stream(
        dict(
        time=[tstep],
        display_time=[ts],
        classification=['Other'],
        count=[a]))

    data_b.stream(
        dict(
        time=[tstep],
        display_time=[ts],
        classification=['ETA_Set'],
        count=[b]))

    data_c.stream(
        dict(
        time=[tstep],
        display_time=[ts],
        classification=['ETA_Request'],
        count=[c]))

    data_d.stream(
        dict(
        time=[tstep],
        display_time=[ts],
        classification=['Requirements'],
        count=[d]))
    data_e.stream(
        dict(
        time=[tstep],
        display_time=[ts],
        classification=['Problem'],
        count=[e]))
    data_f.stream(
        dict(
        time=[tstep],
        display_time=[ts],
        classification=['Accessorials'],
        count=[f]))
    data_g.stream(
        dict(
        time=[tstep],
        display_time=[ts],
        classification=['Appt_Set'],
        count=[g]))
    data_h.stream(
        dict(
        time=[tstep],
        display_time=[ts],
        classification=['Appt_Request'],
        count=[h]))


    push_notebook(handle=handle)
    time.sleep(steptimelive)



Loading BokehJS ...